# Visualizing COVID-19 in US states

This notebook uses the Johns Hopkins University COVID-19 resource to plot data on cases, recoveries, and deaths

#### Notes
- As of 3/24/20, "recovered" is no longer maintained, and the data is replaced by new "confirmed" and "deaths" .csv files. "Recovered" plots are removed from this notebook
- Unfortunately, the "global" time series datasets no longer have individual states' data. Need to pull directory of daily reports from github directly and then assemble a new dataframe


### CURRENTLY WORKING ON:
- Chloropleth of states with the highest change in daily caseload and deaths

In [1]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt, seaborn as sns, plotly.express as px
from datetime import datetime, timedelta
import os

# plotly and cufflinks
import cufflinks as cf
import plotly
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
init_notebook_mode(connected=True) # allows jupyter notebook to access the visualizations in java
cf.go_offline() # make cufflinks go offline

import plotly.graph_objects as go

%matplotlib inline

# Plot COVID-19 cases by state

#### Techniques to use
Dictionary to change state names to abbreviations and vice-versa

Pull data directly from github

## Current issues and Next Steps
- Handling of sums of cases is wonky, especially in early phases
    - Currently this is handled by only keeping data from each day that it is reported. This prevents over-counting of cases in the same location; however, it has the drawback of days when fewer municipalities reported new cases not counting the existing cases. **How do I best handle this??**
    - If I don't remove duplicate data, the result is worse, but I haven't figured out why

### Next steps
- Assemble population of states, plot cases and deaths per capita and rates of growth
    
## Analysis notes

### Fixing dates

So far, the best method I've found to sort dates is just to use the date of the file that's imported.
- Using the "date updated" field to sort by dates causes duplicates or multiple counting because of adjustments to the tally
- Drawback to the current method is that adjusted totals aren't considered, so there are a few negative values that I correct to zero.

#### Approaches attempted
- I added a `file_date` column upon import that labels the date from the url. Should be able to combine that plus the "Date Updated" field to make a unique ID for each country and avoid double-counting
    - Idea 1: For each municipality or item, keep the item that has the latest (largest) file_date, remove earlier ones
    - Easier idea (performed): Check the above link, and go visit the file from the "last updated" day - it is probably easiest and quickest to simply remove records that don't match the file_date and then have `connectgaps=True` in the graph
        - **This does not yield accurate data overall. Some of the updates that occur later than the given time are not included, or they are double-counted**
        - *Best approach:* For each *date* (`datetime.date()`), keep the latest `datetime.time()`
        
##### Approach 2
- For each *date* (`datetime.date()`), keep the latest (largest) `datetime()` (includes update time)
- Application of the `format_datetime()` function recursively means that "Last Update" is now properly formatted with date and time
- Function to keep only the largest "Last Update" value for each "Date Updated" -- **This may also prove erroneous**

##### Approach 3
For items with equal `Date Updated`, keep the one with highest `file_date` value
- This has the same problems as just using `Date Updated` field

### Keeping track of data formatting

Data formatting has changed a bit over time.

#### Dates
- Early formatting date `M/DD/YY HH:MM` or `MM/DD/YYYY HH:MM`. Later dates resumed this format
- Middle dates adopted `YYYY-MM-DDTHH:MM:SS`

#### Locations
- `admin2` refers to county/municipality level location data
- "Combined_Key" is not needed for my analysis, it's an amalgam of place names
- "Country/Region" changed with "Country_Region" in later timepoints
- "Province/State" changed to "Province_State"
- "Latitude" --> "Lat"
- "Longitude" --> "Long_"

__________

### Helper functions to handle dates
Date format for each spreadsheet of daily cases has changed several times. Methods here were made to handle the different formats and make them compatible

In [2]:
### make a method to generate a list of date strings

# This cell is for testing methods before incorporating into functions

# make a single date from today - 60 days
t = datetime.today().date()
d = timedelta(days=60)
print('default t-d string output:', str(t - d))

# reformat t-d string to be mm-dd-yyyy
print('formatted using strftime')
print((t-d).strftime("%m-%d-%Y"))

# Iterator that creates list of strings of the previous [days] dates
days = 10
date_iter = [(datetime.today().date() - timedelta(days=d)).strftime("%m-%d-%Y") 
             for d in range(days, 0, -1)]
date_iter # show the test output

default t-d string output: 2020-07-17
formatted using strftime
07-17-2020


['09-05-2020',
 '09-06-2020',
 '09-07-2020',
 '09-08-2020',
 '09-09-2020',
 '09-10-2020',
 '09-11-2020',
 '09-12-2020',
 '09-13-2020',
 '09-14-2020']

In [3]:
def format_date(s):
    """This function formats the string dates as python datetime objects, returns only dates"""
    if "T" in s:
        date = datetime.strptime(s.split('T')[0], '%Y-%m-%d').date()
    elif "/" in s:
        try:
            date = datetime.strptime(s.split(" ")[0], '%m/%d/%Y').date()
        except:
            date = datetime.strptime(s.split(" ")[0], '%m/%d/%y').date()
    else:
        date = datetime.strptime(s.split(" ")[0], '%Y-%m-%d').date()
    return date

def format_datetime(s):
    """This function formats the string dates as python datetime objects, returns dates and timestamps"""
    if "T" in s:
        date = datetime.strptime(s, '%Y-%m-%dT%H:%M:%S')
    elif "/" in s and len(s.split(' ')[1]) <= 5:
        try:
            date = datetime.strptime(s, '%m/%d/%Y %H:%M')
        except:
            date = datetime.strptime(s, '%m/%d/%y %H:%M')
    elif "/" in s and len(s.split(' ')[1]) > 5: # likely to capture those containing seconds
        try:
            date = datetime.strptime(s, '%m/%d/%Y %H:%M:%S')
        except:
            date = datetime.strptime(s, '%m/%d/%y %H:%M:%S')
    else:
        date = datetime.strptime(s, '%Y-%m-%d %H:%M:%S')
    return date

# tests for formatting
#print('Testing format_date()')
#print(format_date('2020-03-23 23:19:21'))
#print(format_date('2020-03-20T23:19:21'))
#print(format_date('3/22/2020 23:19:21'))

def create_date_list(d, method = 'days'):
    """function that returns a list of formatted date strings 
    based on number of days prior to today start date
    method can be 'days' or 'date'
    'days' = number of days prior to today
    'date' = start date with ending date of today"""
    if method == 'days':
        date_list = [(datetime.today().date() - timedelta(days=days)).strftime("%m-%d-%Y") 
                     for days in range(d, 0, -1)]
    elif method == 'date':
        date_list = [(datetime.today().date() - timedelta(days=d)).strftime("%m-%d-%Y") 
                     for d in range(days, 0, -1)]
    return date_list

#### Manually read .csv files from original JHU repo
1. Create `dateList` which has all inclusive dates for analysis
2. Read csv files from the appropriate URLs corresponding to `dateList`

In [4]:
# create list of dates to read from repo
NUM_DAYS = 90 # input the past number of days of data to import and include

dateList = create_date_list(NUM_DAYS)
suffixes = [s + '.csv' for s in dateList]

# create list of URLs of files to open
main_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'
urls = list(map(lambda x: main_url + x, suffixes))

# Open files and rename discordant columns upon opening
frames = []
for i in urls:
    df = pd.read_csv(i)
    # Add a File_date field based on the url
    df['file_date'] = datetime.strptime(i.split(".")[-2][-10:], '%m-%d-%Y').date()
    # logic to rename fields
    if "Lat" in df.columns:
        df.rename(columns={'Lat':'Latitude'}, inplace=True)
    if "Country_Region" in df.columns:
        df.rename(columns={"Country_Region":"Country/Region"}, inplace=True)
    if "Long_" in df.columns:
        df.rename(columns={"Long_":"Longitude"}, inplace=True)
    if "Province_State" in df.columns:
        df.rename(columns={"Province_State":"Province/State"}, inplace=True)
    if "Last_Update" in df.columns:
        df.rename(columns={'Last_Update':'Last Update'}, inplace=True)
    frames.append(df)
    
data = pd.concat(frames, axis=0, sort=True)

# Convert string dates to datetime objects
data['Date Updated'] = data['Last Update'].apply(format_date)
data['Last Update'] = data['Last Update'].apply(format_datetime)
data.head()

data['file_date'].nunique() # check the number of dates matches with the dateList defined earlier

90

In [5]:
data.head() # check the layout of the states data

,Active,Admin2,Case-Fatality_Ratio,Combined_Key,Confirmed,Country/Region,Deaths,FIPS,Incidence_Rate,Last Update,Latitude,Longitude,Province/State,Recovered,file_date,Date Updated
0,73.0,Abbeville,0.000000,"Abbeville, South Carolina, US",73,US,0,45001.0,297.631182,2020-06-18 04:33:18,34.223334,-82.461707,South Carolina,0,2020-06-17,2020-06-18
1,593.0,Acadia,5.120000,"Acadia, Louisiana, US",625,US,32,22001.0,1007.333387,2020-06-18 04:33:18,30.295065,-92.414197,Louisiana,0,2020-06-17,2020-06-18
2,1004.0,Accomack,1.375246,"Accomack, Virginia, US",1018,US,14,51001.0,3150.142344,2020-06-18 04:33:18,37.767072,-75.632346,Virginia,0,2020-06-17,2020-06-18
3,964.0,Ada,2.231237,"Ada, Idaho, US",986,US,22,16001.0,204.739746,2020-06-18 04:33:18,43.452658,-116.241552,Idaho,0,2020-06-17,2020-06-18
4,12.0,Adair,0.000000,"Adair, Iowa, US",12,US,0,19001.0,167.785235,2020-06-18 04:33:18,41.330756,-94.471059,Iowa,0,2020-06-17,2020-06-18


#### Cleaning Province/State data
1. Manually fix any small errors (eg: 'Chicago')
1. Convert to full state names by `abbrev_us_state`
2. Drop cruise ships not assigned to a state
3. Drop territories
4. Rename things that need renaming

In [6]:
# United States of America Python Dictionary to translate States,
# Districts & Territories to Two-Letter codes and vice versa.
# https://gist.github.com/rogerallen/1583593

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

# thank you to @kinghelix and @trevormarburger for this idea
abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

# Simple test examples
if __name__ == '__main__':
    print("Wisconsin --> WI?", us_state_abbrev['Wisconsin'] == 'WI')
    print("WI --> Wisconsin?", abbrev_us_state['WI'] == 'Wisconsin')
    
# take the ones with abbreviations and convert to state names using abbrev_us_state
def convert_to_state(s):
    s = s.split(',')[-1].strip()
    
    # fix D.C. abbreviation
    if s == "D.C.":
        s = "DC"
    
    if len(s) == 2 and s in abbrev_us_state:
        return abbrev_us_state[s]
    elif '(' in s and len(s.split(' ')[0]) == 2: # has parenthesis and state abbrev (cruise ship)
        return abbrev_us_state[s.split(' ')[0]]
    else:
        return s
    
## Filter and clean dataframe

# drop unneeded columns
#drop_cols = ['Combined_Key', 'Last Update', 'Latitude', 'Longitude', 'Country/Region', 'Admin2', 'file_date']
drop_cols = ['Combined_Key', 'Latitude', 'Longitude', 'Country/Region', 'Admin2']

states = data[data['Country/Region'] == 'US'].drop(drop_cols, axis=1)

# Reorder columns for easier reading
newcols = ['Province/State', 'FIPS', 'file_date','Date Updated', 'Active', 'Confirmed', 'Deaths', 'Recovered']
states = states[newcols]

# Rename "Date Updated" to "Date"
states.rename(columns={'file_date':'Date'}, inplace=True)

# Create 'State' column
states['State'] = states['Province/State'].apply(convert_to_state)

# Drop territories & cruises
to_remove = ['Virgin Islands', 'United States Virgin Islands', 'Guam', 'Puerto Rico', 'US',
            'Diamond Princess', 'Grand Princess', 'Unassigned Location (From Diamond Princess)',
            'Grand Princess Cruise Ship', 'Northern Mariana Islands', 'Recovered',
            'American Samoa', 'U.S.', 'Wuhan Evacuee'] # define territories to drop
states = states[~states['State'].isin(to_remove)] # drop territories

print('Number of states:', states['State'].nunique()) # check number of states

states.head()

Wisconsin --> WI? True
WI --> Wisconsin? True
Number of states: 51


,Province/State,FIPS,Date,Date Updated,Active,Confirmed,Deaths,Recovered,State
0,South Carolina,45001.0,2020-06-17,2020-06-18,73.0,73,0,0,South Carolina
1,Louisiana,22001.0,2020-06-17,2020-06-18,593.0,625,32,0,Louisiana
2,Virginia,51001.0,2020-06-17,2020-06-18,1004.0,1018,14,0,Virginia
3,Idaho,16001.0,2020-06-17,2020-06-18,964.0,986,22,0,Idaho
4,Iowa,19001.0,2020-06-17,2020-06-18,12.0,12,0,0,Iowa


In [7]:
# save dataframes
states.to_csv('states_assembled.csv')
data.to_csv('raw_states_data_assembled.csv')

## Plot state data

In [8]:
today = str(datetime.today().strftime("%m/%d/%Y")) # print string of today's date

# group by state
grouped_states = states.drop(['Province/State', 'FIPS'], axis=1).groupby(
    by=['State', 'Date']).sum()

# fill in active cases
grouped_states['Active'] = grouped_states['Confirmed'] - grouped_states['Deaths'] - grouped_states['Recovered']

grouped_states.head()

Active  Confirmed  Deaths  Recovered
State   Date                                            
Alabama 2020-06-17   26522      27312     790          0
        2020-06-18   27396      28206     810          0
        2020-06-19   28180      29002     822          0
        2020-06-20   28711      29549     838          0
        2020-06-21   29182      30021     839          0

In [9]:
# plot total confirmed cases
states_confirmed = grouped_states.reset_index().pivot(index='Date', columns='State', values='Confirmed').iplot(
    kind='lines', yaxis_type='log', theme='ggplot', connectgaps=True, asFigure=True,
    title='COVID-19 Confirmed Cases by State ({})'.format(today), yaxis_title='Number of Cases', xaxis_title='Date')

states_confirmed

In [10]:
# plot total deaths by state
states_deaths = grouped_states.reset_index().pivot(index='Date', columns='State', values='Deaths').iplot(
    kind='lines', yaxis_type='log', theme='ggplot', connectgaps=True, asFigure=True,
    title='COVID-19 Deaths by State ({})'.format(today), yaxis_title='Number of Cases', xaxis_title='Date')

states_deaths

In [11]:
# plot active cases by state
states_active = grouped_states.reset_index().pivot(index='Date', columns='State', values='Active').iplot(
    kind='lines', yaxis_type='log', theme='ggplot', connectgaps=True, asFigure=True,
    title='Active COVID-19 Cases by State ({})'.format(today), yaxis_title='Number of Cases', xaxis_title='Date')

states_active

In [12]:
# save figures
plotly.io.write_html(states_confirmed, file='states-confirmed.html')
plotly.io.write_html(states_deaths, file='states-deaths.html')

________
## Plot number of new cases for each day
1. Format data using `pivot()` to get an arrangement like this:

|    | state1 | state2 | state3 | .....

date | xxx    | xxx    | xxx    | xxxx

In [13]:
# make DataFrames of daily cases and deaths with rows=date, cols=state
states_daily_confirmed = grouped_states.reset_index().pivot(index='Date', columns='State', values='Confirmed').diff()
states_daily_confirmed[states_daily_confirmed < 0] = 0 # replace all negative values with 0

states_daily_deaths = grouped_states.reset_index().pivot(index='Date', columns='State', values='Deaths').diff()
states_daily_deaths[states_daily_deaths < 0] = 0 # replace all negative values with 0

In [14]:
def select_dates(first, last):
    """Select an inclusive range of dates from a dataframe"""
    dates = [pd.to_datetime(first).date(),
            pd.to_datetime(last).date()]
    return dates

first, last = select_dates('2020-04-21', '2020-04-25') # input dates here

# TEST filtering dataframe for dates
test_dates = states[(states['Date'] >= first)
                   & (states['Date'] <= last)]

test_dates.info() # print info of filtered dateframe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 9 columns):
Province/State    0 non-null object
FIPS              0 non-null float64
Date              0 non-null object
Date Updated      0 non-null object
Active            0 non-null float64
Confirmed         0 non-null int64
Deaths            0 non-null int64
Recovered         0 non-null int64
State             0 non-null object
dtypes: float64(2), int64(3), object(4)
memory usage: 0.0+ bytes


## Plot line graphs of new daily case loads and deaths

Plot daily cases

In [15]:
today = str(datetime.today().strftime("%m/%d/%Y")) # print string of today's date

states_daily_confirmed.iplot(kind='lines', theme='ggplot', connectgaps=True, asFigure=True,
    title='New COVID-19 Cases by State', yaxis_title='Number of Cases', xaxis_title='Date')

In [16]:
window_size = 7
avg_cases = states_daily_confirmed.rolling(window=window_size, min_periods=3).mean()
states_confirmed_plot = avg_cases.iplot(scale='log', yaxis_title='Number of cases', xaxis_title='Date',
              title='Daily New Confirmed Cases by State (Rolling {} Day Average, {})'.format(window_size, today), asFigure = True)

states_confirmed_plot

Plot daily deaths

In [17]:
window_size = 7
avg_deaths = states_daily_deaths.rolling(window=window_size, min_periods=3).mean()
states_deaths_plot = avg_deaths.iplot(scale='log', yaxis_title='Number of deaths', xaxis_title='Date',
              title='Daily New Deaths by State (Rolling {} Day Average, {})'.format(window_size, today), asFigure = True)

states_deaths_plot

In [18]:
# save figures
plotly.io.write_html(states_confirmed_plot, file='states-daily-cases.html')
plotly.io.write_html(states_deaths_plot, file='states-daily-deaths.html')

Get specifics for dates, this will be used in making choropleth

## Next steps
Assemble population of states, plot cases and deaths per capita and rates of growth

## Normalize data by population
Import `csv` of state populations, use it to create "/100k" daily charts and choropleth to better compare states

In [19]:
# open state populations, only use "State" and "Pop" (current population) columns
state_pops = pd.read_csv('state_pops.csv', usecols=['State', 'Pop']).set_index('State')
state_pops.head()

,Pop
State,
California,39937489
Texas,29472295
Florida,21992985
New York,19440469
Pennsylvania,12820878


In [20]:
# normalize state daily cases by state population (/100k)
normalized_confirmed = states_daily_confirmed.copy()
normalized_deaths = states_daily_deaths.copy()

# get rate of infections per 100k people
for col in normalized_confirmed.columns:
    pop = state_pops.loc[col][0]
    normalized_confirmed[col] = (normalized_confirmed[col]/pop)*100000
    
# get rate of deaths per 100k people
for col in normalized_deaths.columns:
    pop = state_pops.loc[col][0]
    normalized_deaths[col] = (normalized_deaths[col]/pop)*100000

### Plot normalized rates

In [21]:
window_size = 7
avg_cases = normalized_confirmed.rolling(window=window_size, min_periods=3).mean()
normalized_confirmed_plot = avg_cases.iplot(scale='log', yaxis_title='Number of cases', xaxis_title='Date',
              title='Daily New Confirmed Cases per 100k by State (Rolling {} Day Average, {})'.format(window_size, today), asFigure = True)

normalized_confirmed_plot

In [22]:
window_size = 7
avg_deaths = normalized_deaths.rolling(window=window_size, min_periods=3).mean()
normalized_deaths_plot = avg_deaths.iplot(scale='log', yaxis_title='Number of cases', xaxis_title='Date',
              title='Daily Deaths per 100k by State (Rolling {} Day Average, {})'.format(window_size, today), asFigure = True)

normalized_deaths_plot

In [23]:
# save normalized figures
plotly.io.write_html(normalized_confirmed_plot, file='states-daily-cases_normalized.html')
plotly.io.write_html(normalized_deaths_plot, file='states-daily-deaths_normalized.html')

## Create choropleth maps
#### Map design
4 maps:
1. Mean new daily cases per capita
2. % change in daily cases from the previous week average (days -14 to -7)
    - Can take this data directly from `normalized_daily_cases`
3. Mean new daily deaths per capita
4. % Change in daily deaths from previous week average
    - Can take this data directly from `normalized_daily_cases`
    
*For all maps, have mouseover data show:*
1. Total daily cases/deaths
2. Total daily cases/deaths per capita
3. % change from previous week

`plotly.express.choropleth(data_frame=None, lat=None, lon=None, locations=None, locationmode=None, geojson=None, featureidkey=None, color=None, hover_name=None, hover_data=None, custom_data=None, animation_frame=None, animation_group=None, category_orders={}, labels={}, color_discrete_sequence=None, color_discrete_map={}, color_continuous_scale=None, range_color=None, color_continuous_midpoint=None, projection=None, scope=None, center=None, title=None, template=None, width=None, height=None)`

### Construct new dataframe
**Columns**: 
    1. State
    2. average new cases
    3. previous week average new cases
    4. percent change in cases
    5. average new deaths
    6. previous week average deaths
    7. percent change in deaths
**Rows** = states


In [24]:
# merge State, average cases/deaths into new dataframe using an inner join
trends = pd.merge(avg_cases.iloc[[-1, -8]].transpose().reset_index(),
                 avg_deaths.iloc[[-1, -8]].transpose().reset_index(),
                 on='State')
trends.columns = ['State name', 'Average new cases', 'Previous week new cases', 'Average new deaths', 'Previous week new deaths']
trends.head()

,State name,Average new cases,Previous week new cases,Average new deaths,Previous week new deaths
0,Alabama,18.876410,20.124942,0.229916,0.273571
1,Alaska,10.704525,10.315270,0.038926,0.097314
2,Arizona,5.345651,7.994276,0.199421,0.371737
3,Arkansas,23.033900,21.167684,0.394867,0.521788
4,California,8.645296,10.190926,0.243595,0.263269


In [25]:
# add percent change columns and reorder
trends['Percent change new cases'] = (trends['Average new cases']/trends['Previous week new cases'])*100-100
trends['Percent change new deaths'] = (trends['Average new deaths']/trends['Previous week new deaths'])*100-100
trends.head()

# reorder columns
trends = trends[['State name', 'Previous week new cases', 'Average new cases', 'Percent change new cases',
                'Previous week new deaths', 'Average new deaths', 'Percent change new deaths']]

# rename states to abbreviations
trends['State'] = trends['State name'].apply(lambda x: us_state_abbrev[x])

trends.head()

,State name,Previous week new cases,Average new cases,Percent change new cases,Previous week new deaths,Average new deaths,Percent change new deaths,State
0,Alabama,20.124942,18.876410,-6.203905,0.273571,0.229916,-15.957447,AL
1,Alaska,10.315270,10.704525,3.773585,0.097314,0.038926,-60.000000,AK
2,Arizona,7.994276,5.345651,-33.131509,0.371737,0.199421,-46.354167,AZ
3,Arkansas,21.167684,23.033900,8.816345,0.521788,0.394867,-24.324324,AR
4,California,10.190926,8.645296,-15.166725,0.263269,0.243595,-7.472826,CA


In [26]:
# create choropleth 1
new_cases_map = px.choropleth(data_frame=trends, locations='State', locationmode='USA-states', scope='usa', color='Average new cases',
             color_continuous_scale='plasma', hover_name='State name', hover_data = ['Average new cases', 'Percent change new cases'],
             title='Daily New Confirmed Cases per 100k Residents (Rolling {} Day Average, {})'.format(window_size, today))

new_cases_map

In [27]:
# percent change in daily new cases rate
# create choropleth 2
change_cases_map = px.choropleth(data_frame=trends, locations='State', locationmode='USA-states', scope='usa', color='Percent change new cases',
             color_continuous_scale='plasma', color_continuous_midpoint = 0, range_color=[-50, 50],
             hover_name='State name', hover_data = ['Percent change new cases', 'Average new cases'],
             title='Percent change in average new cases over the past 7 days (Updated {})'.format(today))

change_cases_map

In [28]:
# plot deaths
# create choropleth 3
new_deaths_map = px.choropleth(data_frame=trends, locations='State', locationmode='USA-states', scope='usa', color='Average new deaths',
             color_continuous_scale='plasma', hover_name='State name', hover_data = ['Average new deaths', 'Average new cases'],
             title='Daily Deaths per 100k Residents (Rolling {} Day Average, {})'.format(window_size, today))

new_deaths_map

In [29]:
# plot total deaths

### TODO: Pull total deaths and total cases data and assemble into a dataframe

# create choropleth 3
total_deaths_map = px.choropleth(data_frame=trends, locations='State', locationmode='USA-states', scope='usa', color='Average new deaths',
             color_continuous_scale='plasma', hover_name='State name', hover_data = ['Average new deaths', 'Average new cases'],
             title='Daily Deaths per 100k Residents (Rolling {} Day Average, {})'.format(window_size, today))

In [30]:
grouped_states.head()

Active  Confirmed  Deaths  Recovered
State   Date                                            
Alabama 2020-06-17   26522      27312     790          0
        2020-06-18   27396      28206     810          0
        2020-06-19   28180      29002     822          0
        2020-06-20   28711      29549     838          0
        2020-06-21   29182      30021     839          0

In [31]:
# save choropleths
plotly.io.write_html(new_cases_map, file='map_states-daily-cases.html') # interactive
plotly.io.write_image(new_cases_map, file='map_states-daily-cases.png')# static
plotly.io.write_html(new_deaths_map, file='map_states-daily-deaths.html')
plotly.io.write_html(change_cases_map, file='map_states-change-cases.html')
plotly.io.write_image(change_cases_map, file='map_states-change-cases.png')

_____
# Below this line is work in progress not yet incorporated
_____

In [32]:
fig = px.line(grouped_states.reset_index(), x='Date', y='New deaths', color='State', log_y=True,
             title='USA states: Deaths per day')
fig.show()

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['State', 'Date', 'Active', 'Confirmed', 'Deaths', 'Recovered'] but received: New deaths